In [ ]:
# Collection of scripts to parse GISAID sequence data and metadata
# Will be updated

# Import modules
from tika import parser # PDF file parser
import pandas as pd
import glob # Unix style pathname pattern expansion
from Bio import SeqIO
from geopy.geocoders import Nominatim # Module to get geo coordinates
import psycopg2 # Module to access PSQL database
from datetime import date

In [ ]:
# Function to replace spaces to underscores in strings
def replace_spaces(string):
    return '_'.join(string.split(' '))

def remove_spaces(string):
    return '/'.join(string.split(' / '))

# Function to read the GISAID metadata PDF file and filter out unnecessary lines/fields
def read_pdf(path):
    pdf = parser.from_file(path)
    raw_content = pdf['content'].lstrip('\n')
    content_lines = raw_content.split('\n')
    filtered_content = list(filter(lambda x: x != '' and x != 'Virus detail' and x != 'Sample information' 
                          and x != 'Institute information' and x != 'Submitter information' 
                                   and (not x.lower().startswith('updated')) 
                                   and (not x.lower().startswith('note')) 
                                   and (not x.lower().startswith('please note'))
                                   and (not x.lower().startswith('apartado')), content_lines))
    return filtered_content[1:]

In [ ]:
# Process text
def process_content(content):
    
    authors = ''
    modified_content = []
    
    for i in range(len(content)):
        if content[i] == 'Sample ID given by the sample' and content[i+1] == 'provider:':
            content[i] = content[i] + ' ' + content[i+1]
            content.pop(i+1)
        if content[i] == 'Sample ID given by the' and content[i+1] == 'submitting laboratory:':
            content[i] = content[i] + ' ' + content[i+1]
            content.pop(i+1)

        if content[i] == 'Additional location' and content[i+1] == 'information:':
            content[i] = content[i] + ' ' + content[i+1]
            content.pop(i+1)
            
        
        if content[i].startswith('Originating lab'):
            
            while not content[i+1].startswith('Address'):
                content[i] = content[i] + ' ' + content[i + 1]
                content.pop(i+1)
        
        if content[i].startswith('Submitting lab'):
            
            while not content[i+1].startswith('Address'):
                content[i] = content[i] + ' ' + content[i + 1]
                content.pop(i+1)
                
        
        if content[i].startswith('Sample ID given by the sample provider'):
            if not content[i+1].startswith('Submitting lab'):
                content[i] = content[i] + ' ' + content[i + 1]
                content.pop(i+1)
                
        
        if content[i].startswith('Additional location'):
            if not content[i + 1].startswith('information') and not content[i+1].startswith('Gender'):
                content[i] = content[i] + ' ' + content[i + 1]
                content.pop(i+1)
                
        
        if content[i].startswith('Sample ID given by the submitting laboratory'):
            if not content[i+1].startswith('Authors'):
                content[i] = content[i] + ' ' + content[i + 1]
                content.pop(i+1)
                
                
        if content[i].startswith('Comment'):
            
            while not content[i + 1].startswith('Originating'):
                content[i] = content[i] + ' ' + content[i + 1]
                content.pop(i+1)                                             

        if content[i].startswith('Authors'):
            authors = content[i]
            
            while not content[i + 1].startswith('Submitter'):
                authors = authors + ' ' + content[i + 1]
                content.pop(i + 1)
            content[i] = authors
             

        if (content[i].startswith('Address:')) and (not content[i+1].startswith('Sample')):
            if not content[i + 1].startswith('Important note'):
                content[i] = content[i] + ' ' + content[i + 1]
                content.pop(i + 1)
                
                
            while (not content[i + 1].startswith('Important note') and (content[i - 1].startswith('Submission'))):
                content[i] = content[i] + ' ' + content[i + 1]
                content.pop(i + 1)
            
            while (not content[i + 1].startswith('Sample') and (content[i-1].startswith('Originating'))):
                content[i] = content[i] + ' ' + content[i + 1]
                content.pop(i + 1)
                
            while (not content[i + 1].startswith('Sample') and (content[i-1].startswith('Submitting'))):
                content[i] = content[i] + ' ' + content[i + 1]
                content.pop(i + 1)
                        
        if content[i].startswith('Important note'):
            break
        modified_content.append(content[i].split(':'))
    
    return modified_content

In [ ]:
# Function to create a pandas dataframe 
def create_dataframe(content):
    data_dict = {}
    for i in range(len(content)):
        
        if content[i][0] == 'Location':
            content[i][0] = replace_spaces(content[i][0])
            content[i][1] = remove_spaces(content[i][1])
            content[i][1] = replace_spaces(content[i][1])
            content[i][1] = content[i][1].lstrip('_')
        else:
            content[i][0] = replace_spaces(content[i][0])
            content[i][1] = replace_spaces(content[i][1]).lstrip('_')

        if content[i][0] == 'Address':
            if content[i-1][0] == 'Originating_lab':
                content[i][0] = 'Originating_lab_address'
            if content[i-1][0] == 'Submitting_lab':
                content[i][0] = 'Submitting_lab_address'
            if content[i-1][0] == 'Submission_Date':
                content[i][0] = 'Submitter_address'

        if content[i][1]:
            data_dict[content[i][0].lower()] = content[i][1]
        else:
            data_dict[content[i][0].lower()] = 'NA'
    
    df = pd.DataFrame([data_dict], columns=data_dict.keys())
    return df

In [ ]:
# Create a list of dataframes
df_list = []
for filename in glob.glob('EPI*.pdf'):
#     print(filename)
    pdf = read_pdf(filename)
    content = process_content(pdf)
    df = create_dataframe(content)
    df_list.append(df)

In [ ]:
# Concatenate dataframes into one dataframe
data = pd.concat(df_list, sort=False)

In [ ]:
# Location info

# Modify location info for Diamond Princess Cruise patients
data.loc[(data.location == 'Asia/Japan/unknown') & 
         (data.additional_location_information == 'Diamond_Princess_cruise_ship'), 
         'location'] = 'Asia/Japan/Diamond_Princess_cruise'

# Get unique locations
locations = (data['location'].unique())
# Create a coordinate dictionary and initiate geolocator object
coordinates = {}
geolocator = Nominatim(user_agent='covid-locations')

def remove_underscore(string):
    return ' '.join(string.split('_'))

# Get coordinates for each location
def get_coords(locations):
    for location in locations:
        precise_location = ''
        if location == 'North_America/USA/Washington':
            precise_location = 'Washington, Seattle'
        else:
            if location.startswith('North_America'):
                precise_location = remove_underscore(', '.join(location.split('/')[-2:]))
            else:
                precise_location = ' '.join(location.split('/')[-1].split('_'))
                
        # Fix typos and other mistakes/bugs
        if location == 'Asia/Vietnam/Quangning':
            precise_location = 'Vietnam, Quang Ninh'
        if location == 'Asia/Vietnam/Vinhphuc':
            precise_location = 'Vinh Phuc'
            
        if location == 'Europe/England/Nottinghamhisre':
            precise_location = 'Nottinghamshire'
        if location == 'Africa/Nigeria/Lagos':
            precise_location = 'Nigeria, Lagos'
        if location == 'Europe/Belgium/Kaulile':
            precise_location = 'Kaulille, Belgium'
        if location == 'Europe/Spain/LaRioja':
            precise_location = 'La Rioja'
        if location == 'Europe/Spain/BasqueCountry':
            precise_location = 'Basque Country'
        
        latitude = 0
        longitude = 0
        print(location)
        if (not location == 'Asia/Japan/Diamond_Princess_cruise') and (not location == 'North_America/USA/California/Grand_Princess_cruise_ship') and (not location == 'North_America/USA/California/Grand_Princess_Cruise_Ship'):
            loc_coordinates = geolocator.geocode(precise_location)
            latitude = loc_coordinates.latitude
            longitude = loc_coordinates.longitude
        elif location == 'Asia/Japan/Diamond_Princess_cruise':
            latitude = 33.406211
            longitude = 137.80692
        elif location == 'North_America/USA/California/Grand_Princess_cruise_ship' or location == 'North_America/USA/California/Grand_Princess_Cruise_Ship':
            latitude =  36.843282
            longitude = -129
        coordinates[location] = [latitude, longitude]

In [ ]:
# Retrieve data in chuncks to avoid errors if there are too many requests
# First 50 locations
get_coords(locations[:50])

In [ ]:
# 50 - 99
get_coords(locations[50:100])

In [ ]:
# 100 - 150
get_coords(locations[100:150])

In [ ]:
# 150 - 199
get_coords(locations[150:200])

In [ ]:
# 200 - 249
get_coords(locations[200:250])

In [ ]:
# 250 - 299
get_coords(locations[250:300])

In [ ]:
# 300 - N
get_coords(locations[300:])

In [ ]:
# Add coordinate data to the dataframe
for location in coordinates:
    data.loc[data.location == location, 'latitude'] = coordinates[location][0]
    data.loc[data.location == location, 'longitude'] = coordinates[location][1]

In [ ]:
# Add country and city/state/province data
countries = []
cities_states_provinces = []
country_and_city_data = {}
for location in locations:
    loc = location.split('/')
    if len(loc) == 2:
        country = loc[1].lstrip('_').rstrip('_')
        loc_data = {'country': country, 'city/state/province': country}
        country_and_city_data[location] = loc_data
        countries.append(country)
    else:
        country = loc[1].lstrip('_').rstrip('_')
        city_state_province = loc[-1].lstrip('_').rstrip('_')
        countries.append(country)
        cities_states_provinces.append(city_state_province)
        loc_data = {'country': country, 'city/state/province': city_state_province}
        country_and_city_data[location] = loc_data

        
unique_countries = set(countries)
unique_cities_states_provinces = set(cities_states_provinces)

# Country ids
country_ids = {}
i = 1
for country in unique_countries:
    country_ids[country] = i
    i += 1
    
# City/State/Province ids
city_state_province_ids = {}
i = 1
for location in unique_cities_states_provinces:
    city_state_province_ids[location] = i
    i += 1
    

# Add country and city columns
for location in country_and_city_data:
    country = country_and_city_data[location]['country']
    city = country_and_city_data[location]['city/state/province']
    data.loc[data.location == location, 'country'] = country
    data.loc[data.location == location, 'city/state/province'] = city

In [ ]:
# Location info (Compare with current database)

fh = open('db.txt')
db_info = [item.rstrip('\n') for item in fh.readlines()]
fh.close()

# Connect to the database and get location data
conn_string = "host='borreliabase.org' dbname='bb3-dev' user='" + db_info[0] + "'" + "password='" + db_info[1] + "'"
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()
geo_query = 'SELECT * FROM CV_GEO ORDER BY geo_id;'
country_query = 'SELECT * FROM COUNTRY ORDER BY country_id;'

def get_table(query):
    cursor.execute(query)
    records = cursor.fetchall()
    table_description = cursor.description
    
    # Column names
    cols = []
    for column in table_description:
        cols.append(column.name)
        
    # Table data
    data = []
    for row in records:
        data.append(list(row))
    df = pd.DataFrame(data, columns = cols)
    return df


geo_table = get_table(geo_query)
country_table = get_table(country_query)
geo_table['geo_name'] = geo_table['geo_name'].apply(lambda x: ':'.join([item.replace(' ', '_') for item in x.split(':')]))

In [ ]:
# Get country ids from database
cursor.execute(country_query)
records = cursor.fetchall()
country_id_dict = {}
for country in records:
    country_name = replace_spaces(country[1])
    if country_name == 'German':
        country_name = 'Germany'
    country_id_dict[country_name] = country[0]

In [ ]:
# Add country ids to the dataframe/table
for country in country_id_dict:
    country_name = replace_spaces(country)
    data.loc[data.country == country_name, 'country_id'] = country_id_dict[country_name]

In [ ]:
# Create new location table
current_locations = [replace_spaces(loc) for loc in geo_table.geo_name.values]
new_locations = {}
for loc in locations:
    temp = [item.rstrip('_').lstrip('_') for item in loc.split('/')]
    area = temp[0]
    area_id = 0
    
    if area == 'North_American' or area == 'north_america':
        area = 'North_America'
    
    if area == 'North_America':
        area_id = 1
    elif area == 'South_America':
        area_id = 2
    elif area == 'Europe':
        area_id = 3
    elif area == 'Asia':
        area_id = 4
    elif area == 'Africa':
        area_id = 5
    elif area == 'Oceania':
        area_id = 6
    elif area == 'Central_America':
        area_id = 7
    
    geo_name = ':'.join(temp[1:])
    
    # Check for typos and other mistakes
    if geo_name == 'USA:California:Grand_Princess_Cruise_Ship' or geo_name == 'USA:California:Grand_Princess_cruise_ship':
        geo_name = 'Grand_Princess_cruise_ship'
    if geo_name == 'Japan:Diamond_Princess_cruise':
        geo_name = 'Diamond_Princess_cruise'
    if geo_name == 'USA:California:San_Francisco_County':
        geo_name = 'USA:California:San_Francisco'
    if geo_name == 'China:Wuhan':
        geo_name = 'China:Hubei:Wuhan'
    if geo_name == 'China:Guandong:Shenzhen' or geo_name == 'China:Shenzhen':
        geo_name = 'China:Guangdong:Shenzhen'
    if geo_name == 'USA:California:Los_Angeles':
        geo_name = 'USA:California:LA'
    if geo_name == 'China:Chongqinq:Yongchuan':
        geo_name = 'China:Chongqing:Yongchuan'
    if geo_name == 'Hong_Kong':
        geo_name = 'China:Hong_Kong'
    if geo_name == 'England':
        geo_name = 'United_Kingdom:England'
    if geo_name == 'England:Nottinghamhisre':
        geo_name = 'United_Kingdom:England:Nottinghamshire'
    if geo_name == 'Vietnam:Quangning':
        geo_name = 'Vietnam:Quang_Ninh'
    if geo_name == 'Vietnam:Vinhphuc':
        geo_name = 'Vietnam:Vinh_Phuc'
    if geo_name == 'Sapin:Galicia':
        geo_name = 'Spain:Galicia'
    if geo_name == 'Australia:NSW:Sydney':
        geo_name = 'Australia:New_South_Wales:Sydney'
    if geo_name == 'China:Chian:NanChang':
        geo_name = 'China:Jiangxi:NanChang'
    if geo_name == 'USA:New_York:Manhattan'or geo_name == 'USA:New_York_City' or geo_name == 'USA:New_York:Brooklyn':
        geo_name = 'USA:New_York:New_York_City'
    if geo_name == 'China:Guangzhou':
        geo_name = 'China:Guangdong:Guangzhou'
    if geo_name == 'Spain:LaRioja':
        geo_name = 'Spain:La_Rioja'
    if geo_name == 'Belgium:Kaulile':
        geo_name = 'Belgium:Kaulille'
    if geo_name == 'Spain:BasqueCountry':
        geo_name = 'Spain:Basque_Country'
    if geo_name == 'USA:Wisconsin:Dane_county' or geo_name == 'USA:WI:Dane_county':
        geo_name = 'USA:Wisconsin:Dane_County'
    if geo_name == 'Russia:Saint_Petersburg':
        geo_name = 'Russia:SaintPetersburg'
    if geo_name.startswith('England'):
        geo_name = 'United_Kingdom:' + geo_name
    
    if geo_name not in current_locations:
        new_locations[geo_name] = area_id

In [ ]:
cols = ['geo_name', 'country_name', 'latitude', 'longitude', 'country_id', 'area_id', 'geo_id']
new_location_data = []
current_geo_id = max(geo_table.loc[geo_table.geo_id < 500, 'geo_id'].values) + 1

current_na_id = max(country_table.loc[(country_table.country_id < 200) 
                                      & (country_table.country_id > 100), 'country_id'].values) + 1
current_sa_id = max(country_table.loc[(country_table.country_id < 300) 
                                      & (country_table.country_id > 200), 'country_id'].values) + 1
current_eu_id = max(country_table.loc[(country_table.country_id < 400) 
                                      & (country_table.country_id > 300), 'country_id'].values) + 1
current_as_id = max(country_table.loc[(country_table.country_id < 500) 
                                      & (country_table.country_id > 400), 'country_id'].values) + 1
current_af_id = max(country_table.loc[(country_table.country_id < 600) 
                                      & (country_table.country_id > 500), 'country_id'].values) + 1
current_oc_id = max(country_table.loc[(country_table.country_id < 700) 
                                      & (country_table.country_id > 600), 'country_id'].values) + 1
# current_ca_id = max(country_table.loc[(country_table.country_id < 800) 
#                                       & (country_table.country_id > 700), 'country_id'].values) + 1
current_ca_id = 701
locs = list(new_locations.keys())
# for loc in new_locations[:50]:
for loc in locs[:50]:
    
    print(loc, new_locations[loc])
    
#     temp = loc[0].split(':')
    temp = loc.split(':')
    country_name = temp[0]
#     area_id = loc[1]
    area_id = new_locations[loc]
    
    
       
    precise_location = ''
    if len(temp) > 1:
        precise_location = ', '.join(temp[1:])
    else:
        precise_location = temp[0]
        
    
    
    
        
    latitude = 0
    longitude = 0
#     if loc[0] != 'Diamond_Princess_cruise':

    if loc != 'Diamond_Princess_cruise':
        geo_info = geolocator.geocode(remove_underscore(precise_location), language='en')
        latitude = geo_info.latitude
        longitude = geo_info.longitude

    latitude 
    
    country_id = 0
    if country_name in country_id_dict.keys():
        country_id = country_id_dict[country_name]
    else:
        if area_id == 1:
            country_id = current_na_id
            current_na_id += 1
        elif area_id == 2:
            country_id = current_sa_id
            current_sa_id += 1
        elif area_id == 3:
            country_id = current_eu_id
            current_eu_id += 1
        elif area_id == 4:
            country_id = current_as_id
            current_as_id += 1
        elif area_id == 5:
            country_id = current_af_id
            current_af_id += 1
        elif area_id == 6:
            country_id = current_oc_id
            current_oc_id += 1
        elif area_id == 7:
            country_id = current_ca_id
            current_ca_id += 1
            
        
        
    geo_name = loc
    geo_id = current_geo_id
    current_geo_id += 1
        
    new_location_data.append(
        [
            geo_name, 
            country_name, 
            latitude, 
            longitude, 
            country_id, 
            area_id, 
            geo_id
        ])

new_location_df = pd.DataFrame(new_location_data, columns=cols)
new_location_df = new_location_df.sort_values('geo_id')

In [ ]:
for loc in locs[50:]:
    
    print(loc, new_locations[loc])
    
#     temp = loc[0].split(':')
    temp = loc.split(':')
    country_name = temp[0]
#     area_id = loc[1]
    area_id = new_locations[loc]


       
    precise_location = ''
    if len(temp) > 1:
        precise_location = ', '.join(temp[1:])
    else:
        precise_location = temp[0]
    
    if loc == 'France:ARA:Privas':
        precise_location = 'Privas'
    if loc == 'France:ARA:Macon':
        precise_location = 'Macon'
    if loc == 'France:ARA:Venissieux':
        precise_location = 'Venissieux'
    if loc == 'France:ARA:Lyon':
        precise_location = 'Lyon, France'
    if loc == 'France:ARA:Bourg-en-Bresse':
        precise_location = 'Bourg-en-Bresse'
    if loc == 'France:ARA:Saint-Priest':
        precise_location = 'Saint-Priest'
        
    latitude = 0
    longitude = 0
#     if loc[0] != 'Diamond_Princess_cruise':

    if loc != 'Diamond_Princess_cruise':
        geo_info = geolocator.geocode(remove_underscore(precise_location), language='en')
        latitude = geo_info.latitude
        longitude = geo_info.longitude

    latitude 
    
    country_id = 0
    if country_name in country_id_dict.keys():
        country_id = country_id_dict[country_name]
    else:
        if area_id == 1:
            country_id = current_na_id
            current_na_id += 1
        elif area_id == 2:
            country_id = current_sa_id
            current_sa_id += 1
        elif area_id == 3:
            country_id = current_eu_id
            current_eu_id += 1
        elif area_id == 4:
            country_id = current_as_id
            current_as_id += 1
        elif area_id == 5:
            country_id = current_af_id
            current_af_id += 1
        elif area_id == 6:
            country_id = current_oc_id
            current_oc_id += 1
        elif area_id == 7:
            country_id = current_ca_id
            current_ca_id += 1
            
        
        
    geo_name = loc
    geo_id = current_geo_id
    current_geo_id += 1
        
    new_location_data.append(
        [
            geo_name, 
            country_name, 
            latitude, 
            longitude, 
            country_id, 
            area_id, 
            geo_id
        ])

new_location_df = pd.DataFrame(new_location_data, columns=cols)
new_location_df = new_location_df.sort_values('geo_id')

In [ ]:
geo_dict = {}
for index, row in geo_table.iterrows():
    geo_name = row.geo_name
    geo_dict[geo_name] = {
        'geo_id': row.geo_id,
        'area_id': row.area_id,
        'country_id': row.country_id
    }
    
new_geo_dict = {}
for index, row in new_location_df.iterrows():
    geo_name = row.geo_name
    new_geo_dict[geo_name] = {
        'area_id': row.area_id,
        'geo_id': row.geo_id,
        'country_id': row.country_id
    }


    
    
for location in locations:
    geo_name = ':'.join([item.rstrip('_').lstrip('_') for item in location.split('/')[1:]])
    if geo_name == 'China:Wuhan':
        geo_name = 'China:Hubei:Wuhan'
    if geo_name == 'China:Guandong:Shenzhen':
        geo_name = 'China:Guangdong:Shenzhen'
    if geo_name == 'USA:California:Los_Angeles':
        geo_name = 'USA:California:LA'
    if geo_name == 'China:Chongqinq:Yongchuan':
        geo_name = 'China:Chongqing:Yongchuan'
    if geo_name == 'Hong_Kong':
        geo_name = 'China:Hong_Kong'
    if geo_name == 'England':
        geo_name = 'United_Kingdom:England'
        
        
    data.loc[data.location == location, 'geo_name'] = geo_name
    if geo_name in geo_dict.keys(): 
        data.loc[data.location == location, 'geo_id'] = geo_dict[geo_name]['geo_id']
        data.loc[data.location == location, 'area_id'] =  geo_dict[geo_name]['area_id']
        data.loc[data.location == location, 'country_id'] = geo_dict[geo_name]['country_id']
       
    if geo_name in new_geo_dict.keys():
        data.loc[data.location == location, 'geo_id'] = new_geo_dict[geo_name]['geo_id']
        data.loc[data.location == location, 'area_id'] = new_geo_dict[geo_name]['area_id']
        data.loc[data.location == location, 'country_id'] = new_geo_dict[geo_name]['country_id']  

In [ ]:
# Replace non-English characters
def replace_chars(string):
    temp = string.replace('è', 'e').replace('é', 'e').replace('ã', 'a').replace('ç', 'c').replace('à', 'a').replace('è', 'e').replace('â', 'a')
    new_string = temp.replace('ó', 'o').replace('ú', 'u').replace('í', 'i').replace('ñ', 'n').replace('á', 'a').replace('ô', 'o')
    return new_string

cols = [
    'location',
    'additional_location_information',
    'originating_lab',
    'originating_lab_address',
    'submitting_lab',
    'submitting_lab_address',
    'authors',
    'submitter',
    'country',
    'city/state/province',
    'geo_name'
]

for col in cols:
    data[col] = data[col].map(replace_chars)
    
new_location_df['geo_name'] = new_location_df['geo_name'].map(replace_chars)

In [ ]:
data

In [ ]:
# Get current date
current_date = date.today()
file_date = str(current_date.month) + '-' + str(current_date.day)

# Write to file
data.to_excel('gisaid-covid19-' + file_date + '.xlsx', index=False)
data.to_csv('gisaid-covid19-' + file_date + '.tsv', index=False, sep='\t')
new_location_df.to_csv('new-geo-data-' + file_date + '.tsv', sep='\t')

## FASTA files

In [ ]:
records = []
for fasta in glob.glob('EPI*.fasta'):
    fh = open(fasta)
    sequence = SeqIO.read(fh, 'fasta')
    fh.close()
    seq_id = sequence.description.split('|')[-1]
    sequence.id = seq_id
    print(seq_id)
    sequence.name = seq_id
    sequence.description = seq_id
    records.append(sequence)

In [ ]:
SeqIO.write(records, 'covid-' + file_date + '.fasta', 'fasta')